In [1]:
import gpytorch as gp
import joblib
from gpytorchModels import ExactGPModel
import time

stime = time.time()
kernel = gp.kernels.MaternKernel(ard_num_dims=17)
mean = gp.means.ZeroMean()
likelihood = gp.likelihoods.GaussianLikelihood()
likelihood.double()

## configuration
crane_config = "m1l"
parts = ["wf", "wr", "r"]

gps = {}
## init gps
model_parameters = joblib.load("model_parameters.pkl")
data = joblib.load("model_data.pkl")
data = data[crane_config]
model_parameters = model_parameters[crane_config]

def init_gp(part):
    gp_cur = ExactGPModel(data["X"], data[part], likelihood, kernel, mean)
    gp_cur.load_state_dict(model_parameters[part]["state_dict"])
    gp_cur.double()
    gp_cur.eval()
    ntime = time.time()
    gp_cur.load_cache(model_parameters[part]["pred_dict"], data["X"])
    # print(time.time() - ntime)
    return gp_cur
gps = list(map(init_gp, parts))
gps = {part: gp_cur for part, gp_cur in zip(parts,gps)}
print(time.time() - stime)


6.1971986293792725
6.072544813156128
7.636045455932617
19.988747358322144


In [3]:
import torch
class MeanVarModelWrapper(torch.nn.Module):
    def __init__(self, gp):
        super().__init__()
        self.gp = gp

    def forward(self, x):
        output_dist = self.gp(x)
        return output_dist.mean, output_dist.variance


In [4]:
with torch.no_grad(), gp.settings.fast_pred_var(), gp.settings.trace_mode():
    gps["wf"].eval()
    pred = gps["wf"](data["X"][0:1, :])  # Do precomputation
    traced_model = torch.jit.trace(MeanVarModelWrapper(gps["wf"]), data["X"][0:1, :])

TracingCheckError: Tracing failed sanity checks!
ERROR: Graphs differed across invocations!
	Graph diff:
		  graph(%self : ClassType<MeanVarModelWrapper>,
		        %input.1 : Tensor):
		    %2 : ClassType<ExactGPModel> = prim::GetAttr[name="gp"](%self)
		    %3 : ClassType<ScaleKernel> = prim::GetAttr[name="covar_module"](%2)
		    %tensor : Tensor = prim::GetAttr[name="raw_outputscale"](%3)
		    %5 : ClassType<MaternKernel> = prim::GetAttr[name="base_kernel"](%3)
		    %tensor.1 : Tensor = prim::GetAttr[name="raw_lengthscale"](%5)
		    %train_input : Tensor = prim::Constant[value=<Tensor>](), scope: MeanVarModelWrapper # C:\Users\ge67kub\.conda\envs\mars\lib\site-packages\gpytorch\models\exact_gp.py:301:0
		    %8 : Tensor[] = prim::ListConstruct(%train_input, %input.1), scope: MeanVarModelWrapper
		    %9 : int = prim::Constant[value=-2](), scope: MeanVarModelWrapper # C:\Users\ge67kub\.conda\envs\mars\lib\site-packages\gpytorch\models\exact_gp.py:311:0
		    %input : Tensor = ate